In [1]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import pandas as pd
import talib as ta
import importlib
import os
import platform
import psutil
import numpy as np
import BacktestingStrategies.Optimization as opt

start = datetime.datetime.today() + relativedelta.relativedelta(years=-2)
end = datetime.date.today()
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('2558.T', 'yahoo', start, end)  #MAXIS 米国株式(S&P500)上場投信

data = data.astype("double")
print(data)

initial = 1000_000
fname = f"html/plot-"
df_returns = pd.DataFrame()

             High    Low   Open  Close       Volume  Adj Close
Date                                                          
2020-09-25  724.0  715.0  720.0  719.0   26364921.0      719.0
2020-09-28  715.0  703.0  710.0  704.0   45780117.0      704.0
2020-09-29  703.0  686.0  699.0  690.0   66937087.0      690.0
2020-09-30  713.0  691.0  694.0  710.0   60240360.0      710.0
2020-10-02  726.0  700.0  700.0  722.0   72341611.0      722.0
...           ...    ...    ...    ...          ...        ...
2022-09-15  364.0  359.0  361.0  361.0   40341207.0      361.0
2022-09-16  371.0  367.0  368.0  369.0   93123001.0      369.0
2022-09-20  368.0  360.0  362.0  365.0   57320464.0      365.0
2022-09-21  377.0  371.0  372.0  376.0  100938847.0      376.0
2022-09-22  386.0  379.0  383.0  380.0   97677204.0      380.0

[488 rows x 6 columns]


In [2]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [3]:
def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [4]:
import BacktestingStrategies.Strategy_BBDandRSI as btst
importlib.reload(btst)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btst.BBandRSI_WithStopLoss, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
opt_params = {
    "n": range(10, 40, 5),\
    "nu": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "nd": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "upper_bound": range(50, 85, 5),
    "lower_bound": range(10, 45, 5),
    "rsi_window": range(10, 30, 2),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.upper_bound > x.lower_bound),\
}
execute_optimize(bt, opt_params)

Start                     2020-09-25 00:00:00
End                       2022-09-22 00:00:00
Duration                    727 days 00:00:00
Exposure Time [%]                   46.516393
Equity Final [$]                   822092.405
Equity Peak [$]                     1000000.0
Return [%]                         -17.790759
Buy & Hold Return [%]              -47.148818
Return (Ann.) [%]                   -9.621407
Volatility (Ann.) [%]               23.163974
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -40.884446
Avg. Drawdown [%]                  -40.884446
Max. Drawdown Duration      687 days 00:00:00
Avg. Drawdown Duration      687 days 00:00:00
# Trades                                    6
Win Rate [%]                        66.666667
Best Trade [%]                       8.972156
Worst Trade [%]                    -33.412316
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1496,27,121,668.29175,445.0,-334044.45800,-0.334123,2020-11-05,2021-03-25,140 days
1,1703,129,151,390.92555,426.0,59731.78835,0.089722,2021-04-06,2021-05-12,36 days
2,1683,228,251,431.12355,421.0,-17037.93465,-0.023482,2021-09-01,2021-10-06,35 days
3,1780,364,389,397.96020,423.0,44570.84400,0.062920,2022-03-24,2022-04-28,35 days
4,1916,408,422,392.93545,422.0,55687.67780,0.073968,2022-05-31,2022-06-20,20 days
5,2146,444,487,376.85625,383.0,13184.48750,0.016303,2022-07-21,2022-09-22,63 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 355740 configurations.
  output = _optimize_grid()
